### `Классика`

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('ru_core_news_sm')
import numpy as np

In [2]:
df = pd.read_csv('authors.csv')
df # мой автор (Крапивин) - 1, другие - 0.

,text,index
0,Васильевна. – Потому что мы там тоже были. Вал...,1
1,"что? Когда ты считаешь, что надо делать одно, ...",1
2,в окно!.. Не смей!.. На миг он увидел себя в з...,1
3,"книги брюхом и рычишь! Журка опять подумал, чт...",1
4,"– сказал старшина Валохин. – Ну, мне же не ска...",1
...,...,...
9546,мимо… Но вот что только скажу: дай вам бог вся...,0
9547,"тысяч на четыреста с лишком и, пожалуй, более....",0
9548,бездельничанье. Но полюбить на всю жизнь — это...,0
9549,"годами — чем дальше, тем больше. Веришь ли, ми...",0


In [3]:
def preprocess(text):
    # убираем стоп слова
    words_filtered = []
    text = text.lower()
    text = text.replace(r"[!?\'\`\-\—]", " ")
    tok = nltk.tokenize.WhitespaceTokenizer()
    words = tok.tokenize(text)    
    stop = stopwords.words('russian')
    words_filtered = [word for word in words if word not in stop]
    words_filtered = str(words_filtered)
    # лемматизируем
    doc = nlp(words_filtered)
    words_lemma = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
    return words_lemma

In [4]:
print('-' *100)
count = 0
for i in range(len(df)):
    df.loc[i, 'text'] = str(preprocess(df.loc[i, 'text']))
    count += 1
    if count % 95 == 0:
        print('-', end='') # для отслеживания процесса

----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------

In [5]:
df

,text,index
0,"['васильевна', 'валохин', 'брандукова', 'ребят...",1
1,"['считать', 'делать', 'говорить', 'делать', 'г...",1
2,"['окно', 'сметь', 'миг', 'увидеть', 'зеркало',...",1
3,"['книга', 'брюхом', 'рычишь', 'журка', 'подума...",1
4,"['сказать', 'старшина', 'валохин', 'сказать', ...",1
...,...,...
9546,"['сказать', 'дать', 'бог', 'счастие', 'какой',...",0
9547,"['тысяча', 'четыреста', 'лишком', 'разумеется'...",0
9548,"['бездельничанье', 'полюбить', 'жизнь', 'ручат...",0
9549,"['год', 'верить', 'милый', 'представить', 'лиц...",0


In [6]:
df_X = df['text']
df_y = df['index']

In [7]:
# векторизация
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df_X)
final_X = X.toarray()
Y = np.array(df_y)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

MultinomialNB судя по тому, что я нашла, самая распространенная моделька для text classification. <br>
Создана как раз для работы с текстом. Удобна для работы с большими объемами текстовых данных.<br>
Поэтому выбираем её.

In [9]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train) # тренировка

MultinomialNB()

In [10]:
print("Accuracy: ", model.score(X_test, y_test)*100,"%")

Accuracy:  98.60432658757851 %


In [11]:
def predict_class(text):
    preprocessed_text = str(preprocess(text))
    text_vectorized = cv.transform([preprocessed_text])
    prediction = model.predict(text_vectorized)
    return prediction

In [12]:
remark_text = 'Женщина шла наискосок через мост прямо на Равика. Она шла  быстро,  но каким-то  нетвердым шагом. Равик заметил ее лишь  тогда, когда она оказалась почти  рядом.'
prediction0 = predict_class(remark_text)
print(f"Класс текста: {prediction0}")

Класс текста: [0]


In [13]:
my_text = 'Ввожу текст написанный мной для того чтоб оценить точность модельки'
prediction1 = predict_class(my_text)
print(f"Класс текста: {prediction1}")

Класс текста: [0]


In [14]:
krapivin_text = 'Револьвер был итальянской системы «Пикколо». В точности как настоящий. Только вместо медных патронов – стеклянные баллончики со сжатым воздухом. Хлопал он оглушительно, бил пластмассовыми пулями крепко, но застрелиться из него было все-таки невозможно. Я и не пробовал. Вместо этого лежал на тахте и стрелял по фужерам.'
prediction2 = predict_class(krapivin_text)
print(f"Класс текста: {prediction2}")

Класс текста: [1]
